# 建立 AWS CLI 環境


In [1]:
from dotenv import load_dotenv
import os
import boto3

# 載入 .env 檔案中的環境變數
load_dotenv()

# 建立 STS 客戶端
sts_client = boto3.client("sts")

# 呼叫 get_caller_identity 以取得當前 AWS 帳號資訊
response = sts_client.get_caller_identity()
account_id = response["Account"]

print(f"AWS Account ID: {account_id}")

AWS Account ID: 339712730744


自動取得所有角色的名稱和 ARN，並可根據名稱或其他條件選擇合適的角色


In [ ]:
import boto3

# 創建 IAM 客戶端
iam_client = boto3.client("iam")


def list_roles():
    try:
        # 呼叫 IAM 客戶端的 list_roles 方法以取得所有角色
        response = iam_client.list_roles()

        # 儲存所有角色的名稱和 ARN
        roles = {role["RoleName"]: role["Arn"] for role in response["Roles"]}
        return roles
    except Exception as e:
        print(f"無法取得角色列表。錯誤訊息：{e}")
        return None


def find_role_arn(role_keyword, roles):
    # 根據指定的關鍵字篩選角色
    for role_name, role_arn in roles.items():
        if role_keyword in role_name:
            return role_arn
    print(f"無法找到包含關鍵字 '{role_keyword}' 的角色。")
    return None


# 取得所有角色
roles = list_roles()

# 定義要搜尋的角色關鍵字
database_role_keyword = "ComprehendDataAccessRole"
translate_role_keyword = "TranslateDemoRole"

# 找出匹配的角色 ARN
database_access_role_arn = find_role_arn(database_role_keyword, roles)
translate_access_role_arn = find_role_arn(translate_role_keyword, roles)

# 檢查取得的 ARN
print(f"Database Access Role ARN: {database_access_role_arn}")
print(f"Translate Access Role ARN: {translate_access_role_arn}")

篩選出名稱中包含「labbucket」的 S3 bucket


In [ ]:
import boto3

# 創建 S3 客戶端
s3_client = boto3.client("s3")


def list_buckets():
    try:
        # 呼叫 list_buckets 方法以取得所有 bucket
        response = s3_client.list_buckets()

        # 從回應中提取每個 bucket 的名稱，並篩選包含 'labbucket' 的名稱
        bucket_names = [
            bucket["Name"]
            for bucket in response["Buckets"]
            if "labbucket" in bucket["Name"]
        ]
        return bucket_names
    except Exception as e:
        print(f"無法取得 bucket 列表。錯誤訊息：{e}")
        return None


# 取得篩選後的 bucket 名稱
bucket_names = list_buckets()

# 顯示取得的 bucket 名稱
if bucket_names:
    print("包含 'labbucket' 的 Bucket 名稱列表：")
    for name in bucket_names:
        print(name)
else:
    print("沒有找到包含 'labbucket' 的 Bucket。")

Version: 02.14.2023


# Lab 7.1: Implementing a Multilingual Solution

本實驗將使用三種不同的 AWS 服務來建立將音訊檔案翻譯為不同語言的文字的解決方案。

## Introducing the business scenario

本實驗將在多種語言之間翻譯影片。

## Lab steps

1. Amazon Transcribe example
2. Amazon Translate example
3. Amazon Polly example
4. Challenge exercise


## 1. Amazon Transcribe example

1. 本部分使用 boto3 用戶端呼叫 Amazon Transcribe 將音訊檔案轉換為文字，運行範例後，可前往 `Amazon Transcribe` 控制台查看轉錄內容。

2. 轉錄需要幾分鐘才能完成。


In [ ]:
# 用於生成全域唯一識別碼（UUID）
import uuid

# 用於處理 JSON 格式的數據
import json

# 這是 AWS 的 Python SDK，用於與 AWS 服務互動
import boto3

# 從 time 模組匯入 sleep 函數，用於讓程式暫停指定的秒數
from time import sleep

# 創建 S3 和 IAM 客戶端
s3_client = boto3.client("s3")
iam_client = boto3.client("iam")


def list_buckets_with_keyword(keyword):
    try:
        # 呼叫 list_buckets 方法以取得所有 bucket，並篩選包含特定關鍵字的名稱
        response = s3_client.list_buckets()
        bucket_names = [
            bucket["Name"]
            for bucket in response["Buckets"]
            if keyword in bucket["Name"]
        ]
        if bucket_names:
            return bucket_names[0]  # 假設只取第一個符合條件的 bucket 名稱
        else:
            print(f"沒有找到包含 '{keyword}' 的 bucket。")
            return None
    except Exception as e:
        print(f"無法取得 bucket 列表。錯誤訊息：{e}")
        return None


def get_role_arn(role_keyword):
    try:
        # 呼叫 list_roles 方法以取得所有角色
        response = iam_client.list_roles()
        # 遍歷所有角色以篩選出包含指定關鍵字的角色 ARN
        for role in response["Roles"]:
            if role_keyword in role["RoleName"]:
                return role["Arn"]
        print(f"無法找到包含關鍵字 '{role_keyword}' 的角色。")
        return None
    except Exception as e:
        print(f"無法取得角色列表。錯誤訊息：{e}")
        return None


# 動態取得包含 'labbucket' 的 S3 bucket 名稱
bucket = list_buckets_with_keyword("labbucket")

# 動態取得包含 'ComprehendDataAccessRole' 和 'TranslateDemoRole' 的角色 ARN
database_access_role_arn = get_role_arn("ComprehendDataAccessRole")
translate_access_role_arn = get_role_arn("TranslateDemoRole")

# 創建一個 AWS Transcribe 的客戶端，用於呼叫 AWS Transcribe 服務
transcribe_client = boto3.client("transcribe")

# 檢查取得的 bucket 名稱和角色 ARN
print(f"S3 Bucket 名稱: {bucket}")
print(f"Database Access Role ARN: {database_access_role_arn}")
print(f"Translate Access Role ARN: {translate_access_role_arn}")

`test.wav` 範例檔案位於的位置

該文件包含音頻短語 `測試。你好，你好，你好。這是一個測試。測試，測試，測試。`


In [ ]:
media_input_uri = f"s3://{bucket}/lab71/transcribe-sample/test.wav"

### 檢查是否具備權限


In [ ]:
import boto3

s3_client = boto3.client("s3")
try:
    s3_client.head_bucket(Bucket=bucket)
    print("Bucket exists and is accessible.")
except Exception as e:
    print(f"Error accessing bucket: {e}")

使用 `test.wav` 檔案作為輸入，創建並啟動一個 AWS Transcribe 的轉錄作業


In [ ]:
# 標識此次轉錄作業的名稱
job_uuid = uuid.uuid1()
transcribe_job_name = f"transcribe-job-{job_uuid}"
# 轉錄輸出的檔案名稱
transcribe_output_filename = "transcribe_output.txt"

# 啟動轉錄作業
response = transcribe_client.start_transcription_job(
    # 轉錄作業的名稱
    TranscriptionJobName=transcribe_job_name,
    # 轉錄的媒體檔案 URI
    Media={"MediaFileUri": media_input_uri},
    MediaFormat="wav",
    # 音訊中的語言代碼
    LanguageCode="en-US",
    # 轉錄結果的輸出 S3 bucket
    OutputBucketName=bucket,
    # 輸出結果檔案的名稱
    OutputKey=transcribe_output_filename,
)

等待作業完成

檢查 AWS Transcribe 轉錄工作（transcription job）狀態的循環，直到工作完成或失敗為止。


In [ ]:
job = None
while True:
    # 指定轉錄工作狀態，並將結果存放在變數 job 中
    # transcribe_job_name 是轉錄工作的名稱。
    job = transcribe_client.get_transcription_job(
        TranscriptionJobName=transcribe_job_name
    )

    # 檢查轉錄工作狀態
    if job["TranscriptionJob"]["TranscriptionJobStatus"] in ["COMPLETED", "FAILED"]:
        # 如果符合條件，則跳出循環
        break
    print(".", end="")
    sleep(20)

print(job["TranscriptionJob"]["TranscriptionJobStatus"])

使用 `get_transcription_job` 方法呼叫的結果檢索輸出文件。

其中 transcription_file 包含的 URI 是轉錄結果的連結，允許下載或查看轉錄文本。

這裡只是查看，並無作為其他用途。


In [ ]:
transcription_file = job["TranscriptionJob"]["Transcript"]["TranscriptFileUri"]
print(transcription_file)

從 S3 下載文件 `transcribe_output.txt`


In [ ]:
s3_client = boto3.client("s3")
with open(transcribe_output_filename, "wb") as f:
    s3_client.download_fileobj(bucket, transcribe_output_filename, f)

開啟檔案並將內容讀入 JSON 物件


In [ ]:
with open(transcribe_output_filename) as f:
    data = json.load(f)

In [ ]:
data

獲得實際的轉錄


In [ ]:
data["results"]["transcripts"][0]["transcript"]

## 2. Amazon Translate example

本部分使用 boto3 用戶端呼叫 `Amazon Translate` 並將文字檔案從英文轉換為西班牙文。

運行後可轉到 `Amazon Translate` 控制台查看翻譯。

有關作業的翻譯和詳細資訊位於 **批量翻譯** 部分。

包含翻譯的文字檔案將位於您的 S3 儲存桶中。該儲存桶還將有一個 **details** 資料夾，其中包含一個 JSON 文件，其中包含有關翻譯的詳細信息，例如源語言和目標語言。

首先，建立翻譯作業。需要輸入和輸出位置。

請注意，Amazon Translate 可以將相同的文字翻譯成多種目標語言。在此範例中，您將使用西班牙語，其語言代碼為“es”。

使用 ID 來取得狀態，等待作業完成。

本機第一次實作花了 `15` 分鐘


In [ ]:
# 生成全域唯一識別碼
import uuid

# 創建 AWS Translate 的客戶端，用於呼叫 AWS Translate 服務
translate_client = boto3.client(service_name="translate")

# 定義輸入數據的 S3 路徑
input_data = f"s3://{bucket}/lab71/translate-sample"

# 定義轉譯後輸出結果的 S3 路徑
output_data = f"s3://{bucket}"

# 生成一個 UUID，作為此次翻譯作業的唯一識別碼
job_uuid = uuid.uuid1()

# 將 UUID 加入到 "translate-job-" 的前綴中
translate_job_name = f"translate-job-{job_uuid}"

# 啟動文本翻譯作業
translate_job_submission = translate_client.start_text_translation_job(
    # 設定翻譯作業的名稱
    JobName=translate_job_name,
    # 配置輸入數據的參數，包含 S3 路徑和數據類型
    InputDataConfig={"S3Uri": input_data, "ContentType": "text/plain"},
    # 配置輸出數據的參數，指定輸出結果的 S3 路徑
    OutputDataConfig={"S3Uri": output_data},
    # 設定存取 AWS Translate 服務所需的 IAM 角色 ARN，以授權訪問 S3 資源
    DataAccessRoleArn=translate_access_role_arn,
    # 指定源語言的語言代碼，這裡為英文（en）
    SourceLanguageCode="en",
    # 指定目標語言的語言代碼，這裡為西班牙語（es）
    TargetLanguageCodes=["es"],
)

# 從作業提交的回應中提取 Job ID
# 將 ID 儲存到 translate_job_id 變數中
translate_job_id = translate_job_submission["JobId"]

# 使用 ID 來取得狀態，等待作業完成
while True:
    translate_job = translate_client.describe_text_translation_job(
        JobId=translate_job_id
    )
    if translate_job["TextTranslationJobProperties"]["JobStatus"] in [
        "COMPLETED",
        "FAILED",
    ]:
        break
    sleep(20)
    print(".", end="")

print(translate_job["TextTranslationJobProperties"]["JobStatus"])


輸出資料夾的格式是根據帳號和作業 ID 建立的，使用此資訊建立一條路徑。


In [ ]:
# 獲取當前 AWS 帳戶的 ID
account_id = boto3.client("sts").get_caller_identity().get("Account")

# 定義翻譯輸出結果的 S3 路徑
# 使用帳戶 ID、固定的 "TranslateText" 標識、翻譯作業的 ID，組成路徑名稱
translate_output_path = f"{account_id}-TranslateText-{translate_job_id}/"

Amazon Translate 輸出多個檔案，下載 .txt 檔案。


In [ ]:
# 建立 S3 資源對象
s3_resource = boto3.resource("s3")

# 使用指定名稱建立 bucket
my_bucket = s3_resource.Bucket(bucket)

# 過濾 bucket 中以 translate_output_path 作為前綴的物件
for my_bucket_object in my_bucket.objects.filter(
    Prefix=translate_output_path
):
    # 提取每個物件的 key，文件名稱及路徑
    file = my_bucket_object.key

    # 檢查文件是否以 "txt" 結尾，即確認文件格式為文本文件
    if file.endswith("txt"):
        # 去除路徑前綴 translate_output_path，使 file 僅包含文件名稱
        file = file.lstrip(translate_output_path)
        # 去除開頭的 "/" 符號，以獲得乾淨的文件名稱
        file = file.lstrip("/")
        
        # 輸出文件名稱，便於查看下載的文件
        print(file)
        
        # 打開本地文件，準備以二進制寫入方式將文件下載到本地
        with open(file, "wb") as f:
            # 使用 s3_client 將 S3 中的物件下載到本地文件
            s3_client.download_fileobj(bucket, my_bucket_object.key, f)


## 3. Amazon Polly example

使用 Polly 建立西班牙文文字檔案的發聲，運行後打開 S3 儲存桶以查看輸出；輸出是一個 `.mp3` 文件，開啟檔案會聽到 Lucia 的聲音說 `Prueba de prueba, este es una prueba`。


In [ ]:
# 呼叫 Polly 語音合成服務
polly_client = boto3.client("polly")

# 指定要讀取的文件路徑
itemname = "lab71/polly-sample/es.test.txt"

# 取得 bucket 中的特定文件對象
obj = s3_resource.Object(bucket, itemname)

# 取得文件內容，並將二進制數據解碼為 `UTF-8` 編碼的字串
body = obj.get()["Body"].read().decode("utf-8")

# 啟動語音合成作業
response = polly_client.start_speech_synthesis_task(
    # 指定使用標準語音合成引擎
    Engine="standard",
    # 指定輸出音訊格式
    OutputFormat="mp3",
    # 將輸出結果儲存到的 S3 bucket
    OutputS3BucketName=bucket,
    # 傳入要合成語音的文本內容
    Text=body,
    # 指定語音 ID，這裡使用西班牙語的 Lucia 聲音
    VoiceId="Lucia"
)


從回應中提取任務 ID。


In [ ]:
task_id = response["SynthesisTask"]["TaskId"]
print(task_id)

使用任務 ID 檢查作業是否已完成


In [ ]:
while True:
    # 查詢語音合成作業的狀態
    polly_job = polly_client.get_speech_synthesis_task(TaskId=task_id)
    
    # 檢查作業的狀態是否為 `completed` 或 `failed``
    if polly_job["SynthesisTask"]["TaskStatus"] in ["completed", "failed"]:
        break
    
    # 若作業仍在進行中，暫停 20 秒以避免頻繁查詢狀態
    sleep(20)
    # 每20秒檢查一次，假如仍在進行，就會看到輸出一個點 `.`
    print(".", end="")

# 循環結束後會輸出最終的作業狀態 `completed` 或 `failed``
print(polly_job["SynthesisTask"]["TaskStatus"])

下載結果


In [ ]:
# 建立 S3 客戶端
s3_client = boto3.client("s3")

# 定義要下載的文件名稱
polly_output_filename = f"{task_id}.mp3"

# 以二進制寫入模式 ("wb") 打開輸出文件
with open(polly_output_filename, "wb") as f:
    # 將 S3 中指定的 的文件下載至本地
    s3_client.download_fileobj(bucket, polly_output_filename, f)


## 4. Challenge exercise

您的挑戰是從具有英語音訊通道的影片建立翻譯後的音訊檔案。

您可以使用前三個範例中的程式碼作為解決方案的範本。

挑戰影片位於 S3 儲存桶的「lab71/challenge」資料夾中。影片檔名為 sample.mp4。此文件也可在此筆記本實例的「/s3」資料夾中找到。
